In [20]:
import json

# Load the fixtures JSON file
with open("../data/raw/fixtures.json", "r") as f:
    fixtures = json.load(f)

# Extract only played (started=True) matches and create a clean match-level dataset
matches = []
for f in fixtures:
    if f.get("started") and f.get("finished"):
        fixture_id = f["id"]
        gameweek = f["event"]
        team_h = f["team_h"]
        team_a = f["team_a"]
        team_h_score = f["team_h_score"]
        team_a_score = f["team_a_score"]

        matches.append({
            "fixture_id": fixture_id,
            "gameweek": gameweek,
            "team": team_h,
            "opponent": team_a,
            "is_home": True,
            "goals_scored": team_h_score,
            "goals_conceded": team_a_score,
            "clean_sheet": int(team_a_score == 0)
        })

        matches.append({
            "fixture_id": fixture_id,
            "gameweek": gameweek,
            "team": team_a,
            "opponent": team_h,
            "is_home": False,
            "goals_scored": team_a_score,
            "goals_conceded": team_h_score,
            "clean_sheet": int(team_h_score == 0)
        })


# Convert to DataFrame
fixtures_df = pd.DataFrame(matches)
fixtures_df.head()


,fixture_id,gameweek,team,opponent,is_home,goals_scored,goals_conceded,clean_sheet
0,1,1,14,9,True,1,0,1
1,1,1,9,14,False,0,1,0
2,4,1,10,12,True,0,2,0
3,4,1,12,10,False,2,0,1
4,2,1,1,20,True,2,0,1


In [21]:
# Load team form data to get the mapping from team ID to team name
team_form_df = pd.read_csv("../data/processed/team_form_dynamic.csv")

# Create a mapping from ID to name using their position in the team form file (assumes IDs 1-indexed)
team_id_to_name = {i + 1: name for i, name in enumerate(team_form_df["team"].unique())}

# Map team and opponent IDs to names in the fixture DataFrame
fixtures_df["team_name"] = fixtures_df["team"].map(team_id_to_name)
fixtures_df["opponent_name"] = fixtures_df["opponent"].map(team_id_to_name)

# Prepare the team form data for merging
form_data = team_form_df[[
    "team", "att_form_home_norm", "att_form_away_norm", "def_form_home_norm", "def_form_away_norm"
]]

# Rename for merging as team
team_form = form_data.rename(columns=lambda col: f"team_{col}" if col != "team" else "team_name")
# Rename for merging as opponent
opp_form = form_data.rename(columns=lambda col: f"opp_{col}" if col != "team" else "opponent_name")

# Merge form into fixture data
merged_df = fixtures_df.merge(team_form, on="team_name", how="left")
merged_df = merged_df.merge(opp_form, on="opponent_name", how="left")

# Show the merged data
merged_df.head()


,fixture_id,gameweek,team,opponent,is_home,goals_scored,goals_conceded,clean_sheet,team_name,opponent_name,team_att_form_home_norm,team_att_form_away_norm,team_def_form_home_norm,team_def_form_away_norm,opp_att_form_home_norm,opp_att_form_away_norm,opp_def_form_home_norm,opp_def_form_away_norm
0,1,1,14,9,True,1,0,1,Man Utd,Fulham,0.500,0.625468,0.726295,0.647017,0.835,0.123596,0.707614,0.635369
1,1,1,9,14,False,0,1,0,Fulham,Man Utd,0.835,0.123596,0.707614,0.635369,0.500,0.625468,0.726295,0.647017
2,4,1,10,12,True,0,2,0,Ipswich,Liverpool,0.000,0.000000,1.000000,1.000000,1.000,0.749064,0.896122,0.352983
3,4,1,12,10,False,2,0,1,Liverpool,Ipswich,1.000,0.749064,0.896122,0.352983,0.000,0.000000,1.000000,1.000000
4,2,1,1,20,True,2,0,1,Arsenal,Wolves,0.500,0.250936,0.792528,0.741264,0.665,0.498127,0.348995,0.505824


In [24]:
# Clean the form features by selecting home or away versions depending on is_home
merged_df["team_att_form"] = merged_df.apply(
    lambda row: row["team_att_form_home_norm"] if row["is_home"] else row["team_att_form_away_norm"], axis=1)

merged_df["team_def_form"] = merged_df.apply(
    lambda row: row["team_def_form_home_norm"] if row["is_home"] else row["team_def_form_away_norm"], axis=1)

merged_df["opp_att_form"] = merged_df.apply(
    lambda row: row["opp_att_form_away_norm"] if row["is_home"] else row["opp_att_form_home_norm"], axis=1)

merged_df["opp_def_form"] = merged_df.apply(
    lambda row: row["opp_def_form_away_norm"] if row["is_home"] else row["opp_def_form_home_norm"], axis=1)

# Select only the relevant features and target for the regression model
cleaned_model_data = merged_df[[
    "team_name", "opponent_name", "is_home",
    "team_att_form", "team_def_form", "opp_att_form", "opp_def_form",
    "goals_scored", "clean_sheet"
]]

# Show the cleaned dataset
cleaned_model_data.head()


,team_name,opponent_name,is_home,team_att_form,team_def_form,opp_att_form,opp_def_form,goals_scored,clean_sheet
0,Man Utd,Fulham,True,0.500000,0.726295,0.123596,0.635369,1,1
1,Fulham,Man Utd,False,0.123596,0.635369,0.500000,0.726295,0,0
2,Ipswich,Liverpool,True,0.000000,1.000000,0.749064,0.352983,0,0
3,Liverpool,Ipswich,False,0.749064,0.352983,0.000000,1.000000,2,1
4,Arsenal,Wolves,True,0.500000,0.792528,0.498127,0.505824,2,1
